In [27]:
import pandas as pd

train = pd.read_csv('/Users/subhiksha/Documents/Sem 2/AML/Project/spark-2022-stream-1/labels/train.csv')
test = pd.read_csv('/Users/subhiksha/Documents/Sem 2/AML/Project/spark-2022-stream-1/labels/sample_submission.csv')
val = pd.read_csv('/Users/subhiksha/Documents/Sem 2/AML/Project/spark-2022-stream-1/labels/val.csv')

In [28]:
def filter_train_val(data,flag):
    new_data = data[data['class']=='debris']
    list = data['class'].value_counts().index.tolist()
    list.remove('debris')
    
    class_df = pd.DataFrame()
    for class_name in list:
        if flag == 'train':
            Class = data[data['class'] == class_name].head(600)
        else:
            Class = data[data['class'] == class_name].head(200)
        class_df = pd.concat([class_df, Class])
    
    class_df = class_df.sample(frac=1, random_state=42)
    class_df.reset_index(drop=True, inplace=True)
    
    Filtered_data = pd.concat([new_data, class_df])
    Filtered_data = Filtered_data.sample(frac=1, random_state=42)
    Filtered_data.reset_index(drop=True, inplace=True)
    return Filtered_data


Filtered_train  = filter_train_val(train,'train')
Filtered_val = filter_train_val(val,'val')

In [30]:
Filtered_train['filename'] = Filtered_train['filename'].str[:-4]
Filtered_train['filename'] = Filtered_train['filename'] + '_2022.jpg'


Filtered_val['filename'] = Filtered_val['filename'].str[:-4]
Filtered_val['filename'] = Filtered_val['filename'] + '_2022.jpg'


In [31]:
category_to_zero = 'debris'
Filtered_train['class'] = Filtered_train['class'].apply(lambda x: 0 if x == category_to_zero else 1)
Filtered_val['class'] = Filtered_val['class'].apply(lambda x: 0 if x == category_to_zero else 1)

In [36]:
import os
import shutil

def image_transfer(data,flag):
    filenames_data_df = pd.DataFrame() 
    filenames_data_df['filename'] = data['filename']
    
    if (flag == 'train'):
        folder_2022_data = "/Users/subhiksha/Documents/Sem 2/AML/Project/spark-2022-stream-1/train"
        master_data_folder = "/Users/subhiksha/Documents/Sem 2/AML/Project/master_data_1/Train"
    else:
        folder_2022_data = "/Users/subhiksha/Documents/Sem 2/AML/Project/spark-2022-stream-1/val"
        master_data_folder = "/Users/subhiksha/Documents/Sem 2/AML/Project/master_data_1/Val"
    
    for filename in filenames_data_df['filename']:    
        finalName = filename
        filename = filename[:-9]
        filename = filename +'.jpg'
        source_path = os.path.join(folder_2022_data, filename)
        destination_path = os.path.join(master_data_folder, finalName)
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)
    
    print("Images transferred.")


image_transfer(Filtered_train,'train')
image_transfer(Filtered_val,'val')

In [39]:
import numpy as np 

Filtered_train['usage'] = 'train'
Filtered_val['usage'] = 'val'
test['usage'] = 'test'

Filtered_train['year'] = 2022
Filtered_val['year'] = 2022
test['year'] = 2022

Filtered_train['illumination'] = np.nan
Filtered_val['illumination'] = np.nan
test['illumination'] = np.nan
test['bbox'] = np.nan
test['class'] = np.nan

# Define the desired order of column names
desired_order = ['illumination', 'filename', 'bbox', 'year','usage','class']

# Reorder the columns of the DataFrame
Filtered_train = Filtered_train[desired_order]
Filtered_val = Filtered_val[desired_order]
test = test[desired_order]


data_2022 = pd.concat([Filtered_train, Filtered_val, test], ignore_index=True)